In [1]:
from haystack import Document
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.readers import ExtractiveReader
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
import pymupdf4llm


def extractWithImages(pdfPath):
    md_text = pymupdf4llm.to_markdown(pdfPath, write_images=True)
    return md_text

pdf_path="./documents/3.pdf"
dataset = extractWithImages(pdf_path)

documents = [Document(content=dataset, meta={"source": pdf_path})]

model = "bert-base-uncased"

document_store = InMemoryDocumentStore()

indexing_pipeline = Pipeline()

indexing_pipeline.add_component(instance=SentenceTransformersDocumentEmbedder(model=model), name="embedder")
indexing_pipeline.add_component(instance=DocumentWriter(document_store=document_store), name="writer")
indexing_pipeline.connect("embedder.documents", "writer.documents")

indexing_pipeline.run({"documents": documents})

/Users/naman/Documents/groupProject/SimpleApplication/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.
/Users/naman/Documents/groupProject/SimpleApplication/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]


{'writer': {'documents_written': 1}}

In [2]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.readers import ExtractiveReader
from haystack.components.embedders import SentenceTransformersTextEmbedder


retriever = InMemoryEmbeddingRetriever(document_store=document_store)
reader = ExtractiveReader()
reader.warm_up()

extractive_qa_pipeline = Pipeline()

extractive_qa_pipeline.add_component(instance=SentenceTransformersTextEmbedder(model=model), name="embedder")
extractive_qa_pipeline.add_component(instance=retriever, name="retriever")
extractive_qa_pipeline.add_component(instance=reader, name="reader")

extractive_qa_pipeline.connect("embedder.embedding", "retriever.query_embedding")
extractive_qa_pipeline.connect("retriever.documents", "reader.documents")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - reader: ExtractiveReader
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> reader.documents (List[Document])

Try extracting some answers.

In [4]:
query = "which architecture is getting used  ?"
extractive_qa_pipeline.run(
    data={"embedder": {"text": query}, "retriever": {"top_k": 3}, "reader": {"query": query, "top_k": 2}}
)

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


{'reader': {'answers': [ExtractedAnswer(query='what is the name of the project ?', score=0.6380125284194946, data='Yummy Inc', document=Document(id=e0e0cebded82722dc00fa203859c616a5d5c6f91013c35ed4b7d8a7094b8b59a, content: '|Sample Software Architecture Document (version 0.7)|Col2|
   |---|---|
   
   
   # Software Architecture Docum...', meta: {'source': './documents/3.pdf'}, score: 40.024666270246755), context=None, document_offset=ExtractedAnswer.Span(start=9791, end=9800), context_offset=None, meta={}),
   ExtractedAnswer(query='what is the name of the project ?', score=0.6315432190895081, data='The Software Architecture Document', document=Document(id=e0e0cebded82722dc00fa203859c616a5d5c6f91013c35ed4b7d8a7094b8b59a, content: '|Sample Software Architecture Document (version 0.7)|Col2|
   |---|---|
   
   
   # Software Architecture Docum...', meta: {'source': './documents/3.pdf'}, score: 40.024666270246755), context=None, document_offset=ExtractedAnswer.Span(start=7128, end=7162),